In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

# TensorFlow: Convert TFRecords to Parquet files

## TFRecords

[TFRecords](https://www.tensorflow.org/tutorials/load_data/tfrecord) are a popular file format to store data for deep learning training with TensorFlow. It is a "simple format for storing a sequence of binary records". In many cases the dataset is too large for the host memory and the dataset is converted into (multiple) tfrecords file to disk. TensorFlow's ecosystem enables to stream the tfrecords from disk to train the model without requiring to load the full dataset.<br><br>
That sounds great, but there are some disadvantages when working with tabular dataset. TFRecords stores the dataset as key, values. In other domains, such as computer vision, this representation is efficient as the key is `image` and the values are a the pixels. For an RGB image with 200x200 resoultion, there are 120000 (200x200x3) values. In a tabular dataset, a feature is often a single number and therefore, there is a significant overhead for using a key in each example. **In some of our experiments, we experienced that tfrecords can be ~4-5x larger than `parquet` files for the same dataset.**
<br><br>
[Parquet](https://en.wikipedia.org/wiki/Apache_Parquet) is another file format to store data. It is a free and open-source data storage format in the Hadoop ecosystem. Many popular systems, such as Spark or Pandas, support to read and write parquet files. 
<br><br>
We developed [NVTabular Data Loaders](https://nvidia.github.io/NVTabular/main/training/index.html) as a customized data loader, fully operating on the GPU. It reads the data from disk into the GPU memory and prepares the next batch on the GPU. Therefore, we do not have any CPU-GPU communication. Our data loader leverages parquet files to reduce the disk pressure. **In our experiments, we experienced that the native data loader is the bottleneck in training tabular deep learning models and by changing the native data loader to NVTabular Data Loader, we saw a 8-9x speed-up.**

### Convert TFRecords to Parquet files
That is a lot of background information. In many cases, we saw that users have their dataset stored as tfrecords files. In this notebook, we provide a tfrecords to parquet examples. Users can transform their dataset to parquet and be able to experiment with NVTabular data loader.

## Create a Synthetic Dataset

First, we will create a synthetic dataset. Afterwards, we will convert the synthetic data to a tfrecord file. The synthetic dataset contains `continuous features`, `categorical features`, `continuous features in a list with variable length`, `categorical features in a list with variable length` and the `label`.<br><br>
The features of a list have variable length, which are often used in session-based recommender systems. For example, the last page views in a session and sessions have different lengths.

In [2]:
import numpy as np
import pandas as pd

In [3]:
def create_synthetic_df(
    N_CONT_FEATURES, N_CAT_FEATURES, N_CONT_LIST_FEATURES, N_CAT_LIST_FEATURES, N_ROWS
):
    dict_features = {}
    for icont in range(N_CONT_FEATURES):
        dict_features["cont" + str(icont)] = np.random.uniform(-1, 1, size=N_ROWS)
    for icat in range(N_CAT_FEATURES):
        dict_features["cat" + str(icat)] = np.random.choice(list(range(10)), size=N_ROWS)
    for icontlist in range(N_CONT_LIST_FEATURES):
        feature_list = []
        for irow in range(N_ROWS):
            n_elements = np.random.choice(list(range(20)))
            feature_list.append(np.random.uniform(-1, 1, size=n_elements).tolist())
        dict_features["cont_list" + str(icontlist)] = feature_list
    for icatlist in range(N_CAT_LIST_FEATURES):
        feature_list = []
        for irow in range(N_ROWS):
            n_elements = np.random.choice(list(range(20)))
            feature_list.append(np.random.choice(list(range(10)), size=n_elements).tolist())
        dict_features["cat_list" + str(icatlist)] = feature_list
    dict_features["label"] = np.random.choice(list(range(2)), size=N_ROWS)
    df = pd.DataFrame(dict_features)
    return df

We can configure the size of the dataset and numbers of features of the different type. As this is just a example, we use only 20,000 rows.

In [4]:
N_ROWS = 20000
N_CONT_FEATURES = 5
N_CAT_FEATURES = 7
N_CONT_LIST_FEATURES = 2
N_CAT_LIST_FEATURES = 3

In [5]:
df = create_synthetic_df(
    N_CONT_FEATURES, N_CAT_FEATURES, N_CONT_LIST_FEATURES, N_CAT_LIST_FEATURES, N_ROWS
)

We can take a look on the dataset.

In [6]:
df.head()

,cont0,cont1,cont2,cont3,cont4,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cont_list0,cont_list1,cat_list0,cat_list1,cat_list2,label
0,-0.598272,-0.665893,-0.315125,0.702566,-0.323849,5,3,5,2,4,8,3,"[0.5072592597602565, -0.47816687381189427, -0....",[],"[4, 2, 5, 1, 7, 6, 4, 8, 8, 7, 6, 0, 3, 0, 9, ...","[0, 1, 5, 9, 8, 1, 9, 3, 4, 5, 9, 1, 8, 0, 9, ...","[1, 4, 5, 6, 8, 3, 5, 9, 0, 6, 0, 2, 1]",0
1,-0.302763,0.102246,-0.197020,0.157954,-0.108024,9,4,8,7,7,3,6,"[-0.08574677514984685, -0.4996899616827948, -0...","[0.23373279277235692, 0.9775817910065407, 0.14...","[2, 5, 0, 8, 6, 0, 2, 8, 4, 7, 2, 9, 7]","[9, 1, 0, 6, 5, 1, 5, 3, 8, 1, 3, 6, 8, 7, 4]","[6, 2, 5, 9, 4, 7, 4, 8, 7, 0, 7, 2, 6, 5, 3, ...",1
2,0.940949,0.028501,0.774253,0.323579,-0.417690,8,9,7,2,8,2,8,"[-0.21767810959793432, 0.7070469796179955, -0....","[0.8299352235765285, -0.0961479916392276, 0.60...","[2, 4, 5, 9, 4, 0]",[2],"[1, 4, 1, 0, 1, 5, 2, 0, 8, 3]",1
3,-0.649665,0.836329,-0.965379,-0.689815,0.946057,9,4,1,9,2,1,4,"[-0.9109288944751086, 0.017280738609947832, -0...",[0.47963783275236826],"[8, 3, 9, 9, 5, 5, 2, 7, 8, 7, 0, 4, 8, 8, 3]","[3, 9, 8, 1, 6, 3, 4, 2, 1, 2, 3, 2, 7, 6, 1]","[0, 4, 5, 9, 4, 6, 0, 8, 6, 9, 5, 2, 6]",1
4,0.786037,-0.550252,0.824308,-0.051628,0.742293,8,0,6,1,5,9,3,"[-0.9196272581988192, 0.26653671490649056]","[-0.34703584253681274, -0.38656340762419905, -...",[8],"[7, 4]","[2, 6]",1


In [7]:
CONTINUOUS_COLUMNS = ["cont" + str(i) for i in range(N_CONT_FEATURES)]
CATEGORICAL_COLUMNS = ["cat" + str(i) for i in range(N_CAT_FEATURES)]
CONTINUOUS_LIST_COLUMNS = ["cont_list" + str(i) for i in range(N_CONT_LIST_FEATURES)]
CATEGORICAL_LIST_COLUMNS = ["cat_list" + str(i) for i in range(N_CAT_LIST_FEATURES)]
LABEL_COLUMNS = ["label"]

## Convert the Synthetic Dataset into TFRecords

After we created the synthetic dataset, we store it to tfrecords.

In [8]:
import tensorflow as tf

2021-08-31 18:57:50.981790: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [9]:
import os
import multiprocessing as mp
from itertools import repeat


def transform_tfrecords(
    df,
    PATH,
    CONTINUOUS_COLUMNS,
    CATEGORICAL_COLUMNS,
    CONTINUOUS_LIST_COLUMNS,
    CATEGORICAL_LIST_COLUMNS,
    LABEL_COLUMNS,
):
    write_dir = os.path.dirname(PATH)
    if not os.path.exists(write_dir):
        os.makedirs(write_dir)
    file_idx, example_idx = 0, 0
    writer = get_writer(write_dir, file_idx)
    column_names = [
        CONTINUOUS_COLUMNS,
        CATEGORICAL_COLUMNS + LABEL_COLUMNS,
        CONTINUOUS_LIST_COLUMNS,
        CATEGORICAL_LIST_COLUMNS,
    ]
    with mp.Pool(8, pool_initializer, column_names) as pool:
        data = []
        for col_names in column_names:
            if len(col_names) == 0:
                data.append(repeat(None))
            else:
                data.append(df[col_names].values)
        data = zip(*data)
        record_map = pool.imap(build_and_serialize_example, data, chunksize=200)
        for record in record_map:
            writer.write(record)
            example_idx += 1
    writer.close()


def pool_initializer(num_cols, cat_cols, num_list_cols, cat_list_cols):
    global numeric_columns
    global categorical_columns
    global numeric_list_columns
    global categorical_list_columns
    numeric_columns = num_cols
    categorical_columns = cat_cols
    numeric_list_columns = num_list_cols
    categorical_list_columns = cat_list_cols


def build_and_serialize_example(data):
    numeric_values, categorical_values, numeric_list_values, categorical_list_values = data
    feature = {}
    if numeric_values is not None:
        feature.update(
            {
                col: tf.train.Feature(float_list=tf.train.FloatList(value=[val]))
                for col, val in zip(numeric_columns, numeric_values)
            }
        )
    if categorical_values is not None:
        feature.update(
            {
                col: tf.train.Feature(int64_list=tf.train.Int64List(value=[val]))
                for col, val in zip(categorical_columns, categorical_values)
            }
        )
    if numeric_list_values is not None:
        feature.update(
            {
                col: tf.train.Feature(float_list=tf.train.FloatList(value=val))
                for col, val in zip(numeric_list_columns, numeric_list_values)
            }
        )
    if categorical_list_values is not None:
        feature.update(
            {
                col: tf.train.Feature(int64_list=tf.train.Int64List(value=val))
                for col, val in zip(categorical_list_columns, categorical_list_values)
            }
        )
    return tf.train.Example(features=tf.train.Features(feature=feature)).SerializeToString()


def get_writer(write_dir, file_idx):
    filename = str(file_idx).zfill(5) + ".tfrecords"
    return tf.io.TFRecordWriter(os.path.join(write_dir, filename))

We define the output path.

In [10]:
PATH = "/raid/tfrecord-test/"

In [11]:
!rm -rf $PATH
!mkdir $PATH

In [12]:
transform_tfrecords(
    df,
    PATH,
    CONTINUOUS_COLUMNS,
    CATEGORICAL_COLUMNS,
    CONTINUOUS_LIST_COLUMNS,
    CATEGORICAL_LIST_COLUMNS,
    LABEL_COLUMNS,
)

We can check the file.

In [13]:
!ls $PATH

00000.tfrecords


## Convert TFRecords to parquet files

Now, we have a dataset in the tfrecords format. Let's use the `convert_tfrecords_to_parquet` function to convert a tfrecord file into parquet.

In [14]:
import glob

from nvtabular.framework_utils.tensorflow.tfrecords_to_parquet import convert_tfrecords_to_parquet

Let's select all TFRecords in the folder.

In [15]:
filenames = glob.glob(PATH + "/*.tfrecords")

Let's call the `convert_tfrecords_to_parquet`.<br><br>
Some details about the parameters:
* `compression_type` is the compression type of the tfrecords. Options: `""` (no compression), `"ZLIB"`, or `"GZIP"`
* `chunks` defines how many data points per `parquet` file should be saved. It splits a tfrecords into multiple parquet files.
* `convert_lists` defines, if feature lists should be converted into muliple feature columns. Even single dataframe series are 1 dimensional arrays when converted back from tfrecords to parquet.   

In [16]:
convert_tfrecords_to_parquet(
    filenames=filenames, output_dir=PATH, compression_type="", chunks=1000, convert_lists=True
)

2021-08-31 18:58:05.260995: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-08-31 18:58:05.263090: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:0b:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2021-08-31 18:58:05.263118: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-08-31 18:58:05.263162: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-08-31 18:58:05.263196: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2021-08-31 18:58:05.263230: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcufft.

## Let's take a look

We can see that `convert_tfrecords_to_parquet` created multiple files per `tfrecord` depending on the chunk size.

In [17]:
filenames = glob.glob(PATH + "/*.parquet")
filenames

['/raid/tfrecord-test/00000_1.parquet',
 '/raid/tfrecord-test/00000_9.parquet',
 '/raid/tfrecord-test/00000_14.parquet',
 '/raid/tfrecord-test/00000_2.parquet',
 '/raid/tfrecord-test/00000_17.parquet',
 '/raid/tfrecord-test/00000_6.parquet',
 '/raid/tfrecord-test/00000_0.parquet',
 '/raid/tfrecord-test/00000_19.parquet',
 '/raid/tfrecord-test/00000_3.parquet',
 '/raid/tfrecord-test/00000_15.parquet',
 '/raid/tfrecord-test/00000_7.parquet',
 '/raid/tfrecord-test/00000_4.parquet',
 '/raid/tfrecord-test/00000_11.parquet',
 '/raid/tfrecord-test/00000_8.parquet',
 '/raid/tfrecord-test/00000_12.parquet',
 '/raid/tfrecord-test/00000_18.parquet',
 '/raid/tfrecord-test/00000_5.parquet',
 '/raid/tfrecord-test/00000_16.parquet',
 '/raid/tfrecord-test/00000_13.parquet',
 '/raid/tfrecord-test/00000_10.parquet']

If we load the first file, we cann see, that it has the same structure as our original synthetic dataset.

In [18]:
df = pd.read_parquet(filenames[0])
df.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat_list0,cat_list1,cat_list2,cont0,cont1,cont2,cont3,cont4,cont_list0,cont_list1,label
0,6,4,2,1,6,5,8,"[7, 3, 5, 8, 1, 9, 8, 0, 2, 3, 6, 4, 2, 3, 0, ...","[7, 8, 3, 3, 9, 3, 7, 7, 1, 3, 2, 8, 2, 2, 9, ...","[9, 9, 1, 9, 2, 0, 5, 1, 2, 4, 7]",0.616503,-0.562299,0.294924,0.368152,0.460910,"[0.10967133, 0.7335732, -0.32737544, 0.5034231...","[0.37586957, 0.57677925, -0.49405763, -0.38141...",0
1,7,6,5,2,2,7,0,"[7, 0, 6, 2, 6, 1, 8, 4, 8, 4, 3, 3, 7, 1, 5]",[],"[6, 1, 0, 2, 4, 8]",0.356238,-0.996674,0.638569,-0.873461,-0.549512,[0.038814984],"[0.9645752, 0.841208, 0.23322387, -0.000990165...",0
2,7,7,0,3,6,9,8,"[2, 8, 5, 3, 2, 9, 4, 0, 8, 6, 0, 5, 9, 5, 4]","[7, 6, 0, 8, 2, 4, 7, 5]","[3, 0, 8, 9, 5, 4]",-0.639743,-0.815482,-0.808328,-0.968345,0.844720,"[-0.3283896, 0.95472634, -0.8193472, 0.9815109...","[-0.2411973, 0.21095003, -0.4520857, -0.961866...",1
3,6,4,8,9,0,2,7,"[8, 4, 3, 7, 7, 6, 7, 4, 4, 1, 8, 8, 8, 9, 4]","[4, 0, 8, 0]",[],0.465874,-0.870847,0.197097,0.254493,-0.363290,"[-0.88873893, 0.45026976, 0.93019474, 0.345771...","[0.95485276, 0.21282451, 0.32634658, 0.0435548...",0
4,7,2,5,6,0,9,7,"[2, 2, 6, 5, 6, 7, 6, 2, 0, 5, 8]","[1, 4, 2, 9, 6, 3, 2]","[5, 8, 0, 0, 2, 7, 8, 6, 5, 8, 9, 8, 7, 5, 6, ...",0.657215,0.697109,0.743880,-0.172813,-0.301696,"[-0.67113024, 0.15799437, -0.3753272, 0.132746...","[-0.78169054, 0.8858877, 0.10161541, 0.1666716...",1
